# 4 Hyperparamater tuning using Optuna
Optuna is a hyperparameter tuning package that is integrated in PyDFLT. In this notebook we describe how to use it.

In [1]:
import os
import sys

import yaml

path_to_project = os.path.dirname(os.path.abspath("")) + "/"
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("decision-focused-learning-codebase"))))

### Prepare basic config

We first define a base config with the basic parameter configuration. Specifically include parameters here that you want to remain fixed.

In [2]:
yaml_dir = "configs/knapsack.yml"
base_config = yaml.safe_load(open(yaml_dir))  # base_config is a dictionary, so does not have to be loaded from a .yml

for key, value in base_config.items():
    print(f"{key}: {value}")

model: {'name': 'knapsack_continuous', 'seed': 5}
data: {'name': 'knapsack', 'num_data': 500, 'seed': 5}
runner: {'num_epochs': 5, 'use_wandb': False, 'experiment_name': 'tuning', 'experiments_folder': 'results/', 'seed': 5}
problem: {'train_ratio': 0.75, 'val_ratio': 0.15, 'seed': 5}
decision_maker: {'name': 'differentiable', 'learning_rate': 0.05, 'batch_size': 32, 'seed': 5}


### Define search spaces
Now we define the parameters we want to search over. In this example we will search the best `learning_rate` and `batch_size`.

In [3]:
from src.utils.optuna import SearchSpaceConfig

# To conduct a hyperparameter tuning experiment, we first need to define the search spaces
search_space = SearchSpaceConfig(path_to_project + "examples/hparams_search_spaces/test_search_config.yaml")

for key, value in search_space.config.items():  # search_space.config is a nested dictionary
    print(f"{key}: {value}")

Auto-Sklearn cannot be imported.
decision_maker: {'learning_rate': {'type': 'float', 'low': 1e-05, 'high': 0.001, 'log': True}, 'batch_size': {'type': 'int', 'low': 1, 'high': 512, 'log': True}}


### Create pruner
A pruner can be used to prune trials that are not promising. This can greatly reduce the time to find good parameters.

In [4]:
import optuna

pruner = optuna.pruners.MedianPruner(
    n_startup_trials=10,  # Number of trials to run before pruning
    n_warmup_steps=15,  # Number of epochs to wait before pruning
    interval_steps=1,  # Interval between pruning checks
    n_min_trials=1,  # Minimum trials required for pruning
)

### Specify study
Specify study name and the folder where the results are stored. A database.db file will be created in the folder `OUTPUT_DIR`. While different problems/methods can be all inside the same database, it can be convenient to separate it by problem or method.

In [5]:
from src.utils.optuna import create_study

STUDY_NAME = "test_study"  # Note that optuna will continue with an existing study if the study already exists
OUTPUT_DIR = "hparam_optimization_results/"  # Folder where the study database is stored (at OUTPUT_DIR/STUDY_NAME)
os.makedirs(f"{OUTPUT_DIR}/{STUDY_NAME}", exist_ok=True)  # Ensure that the folder exists
study = create_study(
    STUDY_NAME,
    storage_url=f"sqlite:///{OUTPUT_DIR}/{STUDY_NAME}/database.db",
    prunner=pruner,
)

Storage url: sqlite:///hparam_optimization_results//test_study/database.db


[I 2025-10-02 14:46:13,277] Using an existing study with name 'test_study' instead of creating a new one.


### Set up dashboard
We can set up a dashboard that visualizes results when the study is running, opened in the background (using package `optuna-dashboard`). Alternatively open the dashboard through the terminal using: `optuna-dashboard sqlite:///examples/hparam_optimization_results//test_study//database.db`

In [6]:
import socket
import subprocess


def find_free_port():
    with socket.socket() as sock:
        sock.bind(("", 0))
        return sock.getsockname()[1]


port = str(find_free_port())
print(f"Found free port: {port}")

subprocess.Popen(
    [
        "optuna-dashboard",
        "sqlite:///hparam_optimization_results//test_study//database.db",
        "--port",
        port,
    ]
)

print(f"See dashboard here: http://localhost:{port}")

Found free port: 49962
See dashboard here: http://localhost:49962


### Define trial

To run hyperparameters tuning with Optuna, we need to define what a 'trial' looks like. We run each configuration for multiple seeds using `run_trial`. Finally, we define `objective_function`, denoting what we use as evaluation metric. In this case this is the validation objective as returned by running the experiment through `run_trial`.

In [7]:
import inspect

from src.utils.optuna import run_trial

print(inspect.getsource(run_trial))


def objective_fn(trial):
    return run_trial(trial, search_space, base_config, seeds=list(range(3)))

def run_trial(trial, search_space: SearchSpaceConfig, base_config, seeds: list):
    assert len(seeds) > 0, "Provide at least one seed!"

    # Parse Optuna Trial to get a config of the values
    trial_config = search_space.get_trial_config(trial)

    # For more robust evaluation, we run each hyperparameter configuration for multiple seeds (same across different configurations)
    # Make sure to use different seeds for the experiments after
    per_seeds_results = []
    for seed in seeds:
        # Update config based on trial
        config = update_config(base_config=base_config, updates_config=trial_config)

        # Initialize all objects with the seed
        config["seed"] = seed
        for key in config:
            if isinstance(config[key], dict) and "seed" in config[key]:
                config[key]["seed"] = seed

        results = run(config, optuna_trial=trial)
        per_seeds_results.append(results)

    try:
        return np.mean(per_seeds_results)
    except op

### Run tuning
Now we run the hyperparameter tuning

In [8]:
study.optimize(objective_fn, n_trials=3, timeout=None, catch=(Exception,), show_progress_bar=True)

  0%|          | 0/3 [00:00<?, ?it/s]

Generating data using knapsack
Computing optimal decisions for the entire dataset...
Optimal decisions computed and added to dataset.
Computing optimal objectives for the entire dataset...
Optimal objectives computed and added to dataset.
Shuffling indices before splitting...
Dataset split completed: Train=375, Validation=75, Test=50
Problem mode set to: train
Problem mode set to: train
Epoch 0/5: Starting initial validation...
Problem mode set to: validation
Epoch Results:
validation/item_value_mean: 4.3376
validation/abs_regret_mean: 8.9304
validation/rel_regret_mean: 0.4107
validation/select_item_mean: 0.3410
validation/objective_mean: 15.7564
validation/sym_rel_regret_mean: 0.2772
validation/mse_mean: 41.6570
Initial best validation metric (abs_regret): 8.930413246154785
Starting training...
Epoch: 1/5
Problem mode set to: train


Listening on http://127.0.0.1:49962/
Hit Ctrl-C to quit.



Epoch Results:
validation/item_value_mean: 4.3376
validation/abs_regret_mean: 8.9304
train/objective_mean: 17.6441
train/grad_norm_mean: 0.0000
validation/rel_regret_mean: 0.4107
train/rel_regret_mean: 0.3719
train/solver_calls_mean: 308.0000
train/abs_regret_mean: 8.1569
validation/select_item_mean: 0.3410
train/sym_rel_regret_mean: 0.2471
train/loss_mean: 8.1569
validation/objective_mean: 15.7564
validation/sym_rel_regret_mean: 0.2772
validation/mse_mean: 41.6570
Problem mode set to: validation
Epoch Results:
validation/item_value_mean: 4.3303
validation/abs_regret_mean: 9.0297
train/objective_mean: 17.6441
train/grad_norm_mean: 0.0000
validation/rel_regret_mean: 0.4118
train/rel_regret_mean: 0.3719
train/solver_calls_mean: 308.0000
train/abs_regret_mean: 8.1569
validation/select_item_mean: 0.3410
train/sym_rel_regret_mean: 0.2471
train/loss_mean: 8.1569
validation/objective_mean: 15.6571
validation/sym_rel_regret_mean: 0.2781
validation/mse_mean: 41.6694
Validation evaluation (abs_r

/Users/noah/PycharmProjects/PyDFLT/.venv/lib/python3.12/site-packages/optuna/trial/_trial.py:501: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(


...

Epoch Results:
validation/item_value_mean: 4.3196
validation/abs_regret_mean: 7.3488
train/objective_mean: 22.5024
train/grad_norm_mean: 0.0000
validation/rel_regret_mean: 0.2437
train/rel_regret_mean: 0.2641
train/solver_calls_mean: 983.0000
train/abs_regret_mean: 8.8665
validation/select_item_mean: 0.4999
train/sym_rel_regret_mean: 0.1622
train/loss_mean: 8.8665
validation/objective_mean: 21.9798
validation/sym_rel_regret_mean: 0.1498
validation/mse_mean: 36.8448
Problem mode set to: validation
Epoch Results:
validation/item_value_mean: 4.3186
validation/abs_regret_mean: 7.3340
train/objective_mean: 22.5024
train/grad_norm_mean: 0.0000
validation/rel_regret_mean: 0.2437
train/rel_regret_mean: 0.2641
train/solver_calls_mean: 983.0000
train/abs_regret_mean: 8.8665
validation/select_item_mean: 0.4997
train/sym_rel_regret_mean: 0.1622
train/loss_mean: 8.8665
validation/objective_mean: 21.9946
validation/sym_rel_regret_mean: 0.1498
validation/mse_mean: 36.8352
Validation evaluation (abs_r

/Users/noah/PycharmProjects/PyDFLT/.venv/lib/python3.12/site-packages/optuna/trial/_trial.py:501: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(


Epoch Results:
validation/item_value_mean: 4.3186
validation/abs_regret_mean: 7.3340
train/objective_mean: 22.4354
train/grad_norm_mean: 0.0000
validation/rel_regret_mean: 0.2437
train/rel_regret_mean: 0.2646
train/solver_calls_mean: 1208.0000
train/abs_regret_mean: 8.8713
validation/select_item_mean: 0.4997
train/sym_rel_regret_mean: 0.1627
train/loss_mean: 8.8713
validation/objective_mean: 21.9946
validation/sym_rel_regret_mean: 0.1498
validation/mse_mean: 36.8352
Problem mode set to: validation
Epoch Results:
validation/item_value_mean: 4.3175
validation/abs_regret_mean: 7.2544
train/objective_mean: 22.4354
train/grad_norm_mean: 0.0000
validation/rel_regret_mean: 0.2436
train/rel_regret_mean: 0.2646
train/solver_calls_mean: 1208.0000
train/abs_regret_mean: 8.8713
validation/select_item_mean: 0.4995
train/sym_rel_regret_mean: 0.1627
train/loss_mean: 8.8713
validation/objective_mean: 22.0742
validation/sym_rel_regret_mean: 0.1497
validation/mse_mean: 36.8291
Validation evaluation (abs

/Users/noah/PycharmProjects/PyDFLT/.venv/lib/python3.12/site-packages/optuna/trial/_trial.py:501: UserWarning: The reported value is ignored because this `step` 5 is already reported.
  warnings.warn(


Optimal decisions computed and added to dataset.
Computing optimal objectives for the entire dataset...
Optimal objectives computed and added to dataset.
Shuffling indices before splitting...
Dataset split completed: Train=375, Validation=75, Test=50
Problem mode set to: train
Problem mode set to: train
Epoch 0/5: Starting initial validation...
Problem mode set to: validation
Epoch Results:
validation/item_value_mean: 3.8620
validation/abs_regret_mean: 6.3541
validation/rel_regret_mean: 0.2326
validation/select_item_mean: 0.4642
validation/objective_mean: 21.4090
validation/sym_rel_regret_mean: 0.1394
validation/mse_mean: 23.9004
Initial best validation metric (abs_regret): 6.3540873527526855
Starting training...
Epoch: 1/5
Problem mode set to: train
Epoch Results:
validation/item_value_mean: 3.8620
validation/abs_regret_mean: 6.3541
train/objective_mean: 20.6727
train/grad_norm_mean: 0.0000
validation/rel_regret_mean: 0.2326
train/rel_regret_mean: 0.2773
train/solver_calls_mean: 308.0

### Retrieve and save results
The dashboard summarizes the results. Alternatively results can be retrieved and saved as follows.

In [9]:
import optuna

from src.utils.optuna import save_progress

path = f"sqlite:///{OUTPUT_DIR}/{STUDY_NAME}/database.db"
studies = optuna.study.get_all_study_summaries(storage=path)

for study_summary in studies:
    study_name = study_summary.study_name
    study = optuna.load_study(study_name=study_name, storage=path)

    print(
        f"Results study: {study_name}\n"
        f"Completed trials: {len(study.trials)}\n"
        f"Best value: {study.best_value:.4f}\n"
        f"Best parameters: {study.best_params}"
    )

save_progress(study, search_space, OUTPUT_DIR)

Results study: test_study
Completed trials: 36
Best value: 3.5843
Best parameters: {'learning_rate': 0.000479018157702876, 'batch_size': 181}
Best configuration saved to hparam_optimization_results/best_config.yaml
